In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def GetData(file):
    '''
    Parses the data in a `.json` file produced by the Unity simulation into an array for each body

    Returns:
        `timeValues`: Array of time values

        `celestialBodies`: Dictionary of arrays for each celestial body

        `lagrangePoints`: Dictionary of arrays for each lagrange point

        `barycentres` : Dictionary of arrays for each barycentre
    '''
    raw = json.load(file)

    timeValues = []
    celestialBodies = {}
    lagrangePoints = {}
    barycentres = {}
    for entry in raw:
        timeValues.append(entry['time'])
        celestialBodyRaw = entry['celestialBodyData']
        for cb in celestialBodyRaw:
            name = cb['name']
            if (name not in celestialBodies):
                celestialBodies[name] = []
            celestialBodies[name].append(cb)

        lagrangePointRaw = entry['lagrangePointData']
        for lp in lagrangePointRaw:
            name = lp['name']
            if (name not in lagrangePoints):
                lagrangePoints[name] = []
            lagrangePoints[name].append(lp)

        barycentresRaw = entry['barycentreData']
        for b in barycentresRaw:
            name = b['name']
            if (name not in barycentres):
                barycentres[name] = []
            barycentres[name].append(b)

    return timeValues, celestialBodies, lagrangePoints, barycentres

In [ ]:
def GetPositions(arrayOfDictsWithPos):
    '''
    Creates a dictionary of the positions of a celestial body or lagrange point through time
    '''
    positions = {
        'x' : [e['position']['x'] for e in arrayOfDictsWithPos],
        'y' : [e['position']['y'] for e in arrayOfDictsWithPos],
        'z' : [e['position']['z'] for e in arrayOfDictsWithPos]
    }
    return positions

In [ ]:
def TranslateAllPositions(positionsDict, translations):
    '''
    Translates the positions in positionsDict by subtracting the positions in translations
    '''
    translatedPositions = {
        'x': [],
        'y': [],
        'z': []
    }
    for i in range(len(positionsDict['x'])):
        translatedPositions['x'].append(positionsDict['x'][i] - translations['x'][i])
        translatedPositions['y'].append(positionsDict['y'][i] - translations['y'][i])
        translatedPositions['z'].append(positionsDict['z'][i] - translations['z'][i])

    return translatedPositions

In [ ]:
def PlotPositions(positions, label, amountOfPoints = 1800):
    startX = positions['x'][0]
    startZ = positions['z'][0]
    plt.scatter(startX, startZ)
    plt.plot(positions['x'][:amountOfPoints], positions['z'][:amountOfPoints], label=label)

In [ ]:
def PlotLayout(fig, plotTitle, bounds):
    plt.grid()
    plt.legend()
    plt.xlabel('x')
    plt.ylabel('z', rotation=0)
    plt.xlim(-bounds, bounds)
    plt.ylim(-bounds, bounds)
    plt.title(plotTitle)
    fig.show()

def PlotOffsets(fileNames, offsetLabels, plotTitle, centre, amountOfPoints = 1800, combine=True, roundBounds = 100):
    '''
    Plots multiple scenarios with different offsets in one figure
    
    Param:
        `fileNames`: Names of the json files containing the data to plot

        `offsetLabels`: Labels of the different scenarios to put in the legend

        `plotTitle`: Title to give to the plot

        `centre`: String expecting either 'B', to normalize the positions around the barycentre
                  or 'L4', to normalize the positions around the L4 point.
    '''
    isFirst = True
    fig = plt.figure(figsize=(5,5))

    labelIdx = 0
    bounds = 0
    for name in fileNames:
        f = open(name)
        timeValues, celestialBodies, lagrangePoints, barycentres = GetData(f)
        f.close()

        # Get positions of relevant bodies
        leadingObjectPos = GetPositions(celestialBodies['Leading Object'])
        L4Pos = GetPositions(lagrangePoints['Sun-Jupiter L4'])
        baryPos = GetPositions(barycentres['Barycentre'])

        if centre=='B':
            # Normalize positions around barycentre
            leadingObjectPos = TranslateAllPositions(leadingObjectPos, baryPos)
            L4Pos = TranslateAllPositions(L4Pos, baryPos)
        elif centre=='L4':
            # Normalize positions around L4
            leadingObjectPos = TranslateAllPositions(leadingObjectPos, L4Pos)
            L4Pos = TranslateAllPositions(L4Pos, L4Pos)

        # Plot L4 only once
        if isFirst:
            PlotPositions(L4Pos, 'Sun-Jupiter L4', amountOfPoints)
            isFirst = False


        PlotPositions(leadingObjectPos, offsetLabels[labelIdx], amountOfPoints)
        labelIdx+=1

        # Determine maximum position in both directions for both axes to get square grid
        xMin = np.min(np.append(leadingObjectPos['x'][:amountOfPoints], L4Pos['x'][:amountOfPoints]))
        xMax = np.max(np.append(leadingObjectPos['x'][:amountOfPoints], L4Pos['x'][:amountOfPoints]))
        zMin = np.min(np.append(leadingObjectPos['z'][:amountOfPoints], L4Pos['z'][:amountOfPoints]))
        zMax = np.max(np.append(leadingObjectPos['z'][:amountOfPoints], L4Pos['z'][:amountOfPoints]))

        maxPosition = np.max([np.abs(xMin), np.abs(xMax), np.abs(zMin), np.abs(zMax)])
        if roundBounds != 0:
            maxPosition = ((int(maxPosition) / roundBounds) + 1) * roundBounds

        if not combine:
            bounds = maxPosition
            PlotLayout(fig, plotTitle, bounds)
            fig = plt.figure(figsize=(5,5))
            isFirst = True
        else:
            bounds = np.max([bounds, maxPosition])
            
    if combine:
        PlotLayout(fig, plotTitle, bounds)

In [ ]:
PlotOffsets(['Data/Basic_Setup.json'], ['Leading Object'], 'Asteroid position (relative to L4)', 'L4', roundBounds=0.4)

f = open('Data/Basic_Setup.json')
timeValues, celestialBodies, lagrangePoints, barycentres = GetData(f)
f.close()

# Get positions
jupiterPos = GetPositions(celestialBodies['Jupiter'])
leadingObjectPos = GetPositions(celestialBodies['Leading Object'])
sunPos = GetPositions(celestialBodies['Sun'])
baryPos = GetPositions(barycentres['Barycentre'])
L4Pos = GetPositions(lagrangePoints['Sun-Jupiter L4'])

# Normalize all positions around the barycentre
jupiterPos = TranslateAllPositions(jupiterPos, baryPos)
leadingObjectPos = TranslateAllPositions(leadingObjectPos, baryPos)
sunPos = TranslateAllPositions(sunPos, baryPos)
L4Pos = TranslateAllPositions(L4Pos, baryPos)

fig = plt.figure(figsize=(5,5))
PlotPositions(jupiterPos, 'Jupiter')
PlotPositions(L4Pos, 'L4')
PlotPositions(leadingObjectPos, 'Leading Object')
PlotPositions(sunPos, 'Sun')
# PlotPositions(baryPos, 'barycentre')
plt.grid()
plt.legend()
plt.xlabel('x')
plt.ylabel('z', rotation=0)
plt.title('All positions (relative to barycentre)')
fig.show()

In [ ]:
# Plots all mass offset orbits in one plot
# Data file names
massOffsetNames = ['Data/Mass_Offset_10.json', 'Data/Mass_Offset_100.json', 'Data/Mass_Offset_500.json', 'Data/Mass_Offset_1000.json']
# Names for labels in plot
massOffsetLabels = ['10', '100', '500', '1000']

PlotOffsets(massOffsetNames, massOffsetLabels, 'Asteroid position (relative to L4)', 'L4', roundBounds=0.6)
PlotOffsets(massOffsetNames, massOffsetLabels, 'Asteroid and L4 positions (relative to barycentre)', 'B')

In [ ]:
# Plots all X position offsets in one plot
# Data file names
posXOffsetNames = ['Data/Position_Offset_(-30,0,0).json', 'Data/Position_Offset_(-20,0,0).json', 'Data/Position_Offset_(-10,0,0).json', 'Data/Position_Offset_(10,0,0).json', 'Data/Position_Offset_(20,0,0).json', 'Data/Position_Offset_(30,0,0).json']
# Names for labels in plot
plotLabels = ['-30', '-20', '-10', '10', '20', '30']

PlotOffsets(posXOffsetNames, plotLabels, 'Asteroid and L4 positions (relative to barycentre)', 'B', combine=False)
PlotOffsets(posXOffsetNames, plotLabels, 'Asteroid and L4 positions (relative to barycentre)', 'B')
PlotOffsets(posXOffsetNames, plotLabels, 'Asteroid position (relative to L4)', 'L4')


In [ ]:
plotLabels = ['-30', '-20', '-10', '10', '20', '30']
posZOffsetNames = ['Data/Position_Offset_(0,0,-30).json', 'Data/Position_Offset_(0,0,-20).json', 'Data/Position_Offset_(0,0,-10).json', 'Data/Position_Offset_(0,0,10).json', 'Data/Position_Offset_(0,0,20).json', 'Data/Position_Offset_(0,0,30).json']

PlotOffsets(posZOffsetNames, plotLabels, 'Asteroid and L4 positions (relative to barycentre)', 'B')
PlotOffsets(posZOffsetNames, plotLabels, 'Asteroid position (relative to L4)', 'L4', roundBounds=20)

In [ ]:
# Set this for offset in X direction True) or Z direction (False)
useXOffset = True

# Plots all X velocity offsets in one plot
# Data file names
velocityXOffsetNames = ['Data/Velocity_Offset_(-30,0,0).json', 'Data/Velocity_Offset_(-20,0,0).json', 'Data/Velocity_Offset_(-10,0,0).json', 'Data/Velocity_Offset_(10,0,0).json', 'Data/Velocity_Offset_(20,0,0).json', 'Data/Velocity_Offset_(30,0,0).json']
# Names for labels in plot
plotLabels = ['-30', '-20', '-10', '10', '20', '30']

PlotOffsets(velocityXOffsetNames, plotLabels, 'Asteroid and L4 positions (relative to barycentre)', 'B')
PlotOffsets(velocityXOffsetNames, plotLabels, 'Asteroid position (relative to L4)', 'L4', roundBounds=20)


In [ ]:
plotLabels = ['-30', '-20', '-10', '10', '20', '30']
velocityZOffsetNames = ['Data/Velocity_Offset_(0,0,-30).json', 'Data/Velocity_Offset_(0,0,-20).json', 'Data/Velocity_Offset_(0,0,-10).json', 'Data/Velocity_Offset_(0,0,10).json', 'Data/Velocity_Offset_(0,0,20).json', 'Data/Velocity_Offset_(0,0,30).json']

PlotOffsets(velocityZOffsetNames, plotLabels, 'Asteroid and L4 positions (relative to barycentre)', 'B')
PlotOffsets(velocityZOffsetNames, plotLabels, 'Asteroid position (relative to L4)', 'L4')

In [ ]:
# Singular plots

PlotOffsets(['Data/Position_Offset_(10,0,0).json'], ['10'], 'Asteroid with postition offset in X \ndirection relative to the barycentre', 'L4', 72*50)
PlotOffsets(['Data/Velocity_Offset_(0,0,30).json'], ['30'], 'Asteroid and L4 positions (relative to barycentre)', 'B', 72*50)
PlotOffsets(['Data/Mass_Offset_10.json'], ['10'], 'Asteroid with mass offset of ten relative to L4', 'L4', roundBounds=0.6)